**1.**

**Importing necessary libraries and exploring the dataset**

In [1]:
from sklearn.datasets import load_boston
import numpy as np
import matplotlib.pyplot as plt
import warnings
from sklearn.model_selection import KFold
from sklearn.preprocessing import PolynomialFeatures

## Avoid printing out warnings
with warnings.catch_warnings():
     warnings.filterwarnings("ignore")
     X, y = load_boston(return_X_y=True)

**Exploring the Dataset**

In [2]:
print("Number of rows and column in the data:", X.shape)
print("Dimension of label data:", y.shape)

Number of rows and column in the data: (506, 13)
Dimension of label data: (506,)


In [3]:
print("The dataset:\n",X )
print("\n The labeled data:\n",y)

The dataset:
 [[6.3200e-03 1.8000e+01 2.3100e+00 ... 1.5300e+01 3.9690e+02 4.9800e+00]
 [2.7310e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9690e+02 9.1400e+00]
 [2.7290e-02 0.0000e+00 7.0700e+00 ... 1.7800e+01 3.9283e+02 4.0300e+00]
 ...
 [6.0760e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 5.6400e+00]
 [1.0959e-01 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9345e+02 6.4800e+00]
 [4.7410e-02 0.0000e+00 1.1930e+01 ... 2.1000e+01 3.9690e+02 7.8800e+00]]

 The labeled data:
 [24.  21.6 34.7 33.4 36.2 28.7 22.9 27.1 16.5 18.9 15.  18.9 21.7 20.4
 18.2 19.9 23.1 17.5 20.2 18.2 13.6 19.6 15.2 14.5 15.6 13.9 16.6 14.8
 18.4 21.  12.7 14.5 13.2 13.1 13.5 18.9 20.  21.  24.7 30.8 34.9 26.6
 25.3 24.7 21.2 19.3 20.  16.6 14.4 19.4 19.7 20.5 25.  23.4 18.9 35.4
 24.7 31.6 23.3 19.6 18.7 16.  22.2 25.  33.  23.5 19.4 22.  17.4 20.9
 24.2 21.7 22.8 23.4 24.1 21.4 20.  20.8 21.2 20.3 28.  23.9 24.8 22.9
 23.9 26.6 22.5 22.2 23.6 28.7 22.6 22.  22.9 25.  20.6 28.4 21.4 38.7
 43.8 33.2 27.5 26.5 18.

**Making new X since we also need the constant term in the equation, so we add 1s column to left side of the dataset.**  

In [4]:
X_new = np.c_[np.ones((X.shape[0],1)),X]
X_new.shape

(506, 14)

In [5]:
# function to normalize features
def normalize_features(X):
    mean = np.mean(X, axis=0)
    
    std = np.std(X, axis=0)
    
    normalized_X = (X - mean) / std
    
    return normalized_X

**3.** 

**Linear Regression**


In [6]:
#Function to fit LR using closed form solution
def weight_function(X,y):
    weight = np.linalg.inv(X.T @ X) @ X.T @ y
    return weight
    
# Function to compute mean squared error
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

#Function to compute predicted values
def predicted_values(X, weight):
    return X @ weight

#K-Cross Validation for LR
def k_fold_cross_validation(X, y, k, weight_function):
    
    kf = KFold(n_splits=k, shuffle=True,random_state=42)
    
    mse_scores_test_set = []
    mse_scores_train_set = []
    
    for train, test in kf.split(X_new):
        X_train, X_test = X_new[train], X_new[test]
        y_train, y_test = y[train], y[test]

        weight = weight_function(X_train, y_train)

        y_train_pred = predicted_values(X_train, weight)
        y_test_pred = predicted_values(X_test, weight)

        mse_scores_train_set.append(mean_squared_error(y_train, y_train_pred))
        mse_scores_test_set.append(mean_squared_error(y_test, y_test_pred))
    
    return np.mean(mse_scores_test_set), np.mean(mse_scores_train_set)

In [7]:
print("(Mean Square Error for Test Set, Mean Square Error for Train Set):",end="")
k_fold_cross_validation(X_new,y,10,weight_function)

(Mean Square Error for Test Set, Mean Square Error for Train Set):

(23.364203007531508, 21.818586996144017)

**4.**

**Ridge regularization model**

In [8]:
# Closed form solution function for ridge regression
def ridge_weight_function(X,y,lambdaValue):
    X_t_and_X = X.T @ X
    
    A=np.eye(X_t_and_X.shape[1]) 
    A[0,0]=0                    #setting the top left of the identity as 0
    
    weight = np.linalg.inv(X_t_and_X + lambdaValue*A)@ X.T @ y
    return weight

#MSE function as before
def mean_squared_error(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

#Prediction Function as before
def predicted_values(X, weight):
    return X @ weight

#K-Cross_validation for Ridge Regression
def ridge_k_fold_cross_validation(X, y, k, ridge_weight_function, lambdaValue):
    kf = KFold(n_splits=k, shuffle=True,random_state=42)
    
    mse_scores_test_set = []
    mse_scores_train_set = []
    
    for train, test in kf.split(X):
        X_train, X_test = X[train], X[test]
        y_train, y_test = y[train], y[test]

        weight = ridge_weight_function(X_train, y_train, lambdaValue)

        y_train_pred = predicted_values(X_train, weight)
        y_test_pred = predicted_values(X_test, weight)

        mse_scores_train_set.append(mean_squared_error(y_train, y_train_pred))
        mse_scores_test_set.append(mean_squared_error(y_test, y_test_pred))
    
    return np.mean(mse_scores_test_set), np.mean(mse_scores_train_set)

**5**.

**Finding best lambda**

In [9]:
#Function to print lambdas and find best lambda
def best_lambda(X, y, lambdas, k=10):

    best_lambda = None
    best_test_mse = float('inf')

    for lamb in lambdas:
        print("Lambda:",lamb)
        print("(Mean Square Error for Test Set, Mean Square Error for Train Set):",end="\n")
        test_mse, train_mse = ridge_k_fold_cross_validation(X_new,y,10,ridge_weight_function,lamb)
        print(f"{test_mse}, {train_mse}")

        if test_mse < best_test_mse:
            best_test_mse = test_mse
            best_lambda = lamb
            
        print()
        
    print("Best Lambda:", best_lambda, "with MSE:", best_test_mse)
    return best_lambda, best_test_mse

# Lambda values
lambdas = np.logspace(1, 7, num=13)

best_lambda, best_test_mse = best_lambda(X, y, lambdas, k=10)

Lambda: 10.0
(Mean Square Error for Test Set, Mean Square Error for Train Set):
24.151167028806483, 22.609088327776966

Lambda: 31.622776601683793
(Mean Square Error for Test Set, Mean Square Error for Train Set):
24.432328644123732, 22.98794193527157

Lambda: 100.0
(Mean Square Error for Test Set, Mean Square Error for Train Set):
25.20417072442644, 23.860963584131078

Lambda: 316.22776601683796
(Mean Square Error for Test Set, Mean Square Error for Train Set):
26.81563492602445, 25.5229860528301

Lambda: 1000.0
(Mean Square Error for Test Set, Mean Square Error for Train Set):
29.109564651842412, 27.862974452706975

Lambda: 3162.2776601683795
(Mean Square Error for Test Set, Mean Square Error for Train Set):
32.54415106038437, 31.37419291420291

Lambda: 10000.0
(Mean Square Error for Test Set, Mean Square Error for Train Set):
38.47163995070741, 37.37495995816031

Lambda: 31622.776601683792
(Mean Square Error for Test Set, Mean Square Error for Train Set):
47.36787615414065, 46.32456

**6.**

**Polynomial Regression** 

In [10]:
polynomial = PolynomialFeatures(degree=2, include_bias=False)

X_array = np.array(X)

polynomial.fit(X_array)

polynomial_features = polynomial.transform(X_array)

print("Shape of new data :", polynomial_features.shape)

Shape of new data : (506, 104)


In [11]:
X_polynomial = np.c_[np.ones((polynomial_features.shape[0],1)),polynomial_features]

best_test_mse, best_train_mse = ridge_k_fold_cross_validation(X_polynomial,y,10,ridge_weight_function,best_lambda)

print("Performance with Best Lambda:", best_lambda)
print(f"Average test MSE: {best_test_mse}, Average training MSE: {best_train_mse}")


Performance with Best Lambda: 10.0
Average test MSE: 13.894347259785476, Average training MSE: 6.771334220874536


**7.**

**Multivariate Regression using gradient descent**

In [12]:
#Function to perform gradient descent for MVR
def weight_function_gradientdescent(X, y, rate, iterations=100000):
    getwts = np.random.randn(X.shape[1],)
    
    m = X.shape[0]
    
    for iteration in range(iterations):
        gradient = (2/m) * X.T.dot(np.matmul(X,getwts)-y)
        getwts = getwts - rate * gradient
        
    return getwts

def gardientdescent_k_fold_cross_validation(X, y, k, weight_function_gradientdescent,learning_rate):
    m, n = X.shape
    
    X_new = np.c_[np.ones((m, 1)), X]
    
    kf = KFold(n_splits=k,shuffle=True,random_state=42)
    
    mse_scores_test_set = []
    mse_scores_train_set = []
    
    for train, test in kf.split(X):        
        X_train, X_test = X_new[train], X_new[test]
        y_train, y_test = y[train], y[test]
        
        weight = weight_function_gradientdescent(X_new, y,learning_rate)
        
        y_train_pred = predicted_values(X_train, weight)
        y_test_pred = predicted_values(X_test, weight)
        
        mse_scores_train_set.append(mean_squared_error(y_train, y_train_pred))
        mse_scores_test_set.append(mean_squared_error(y_test, y_test_pred))
        
    return np.mean(mse_scores_test_set), np.mean(mse_scores_train_set)

In [13]:
print("(Mean Square Error for Test Set, Mean Square Error for Train Set):",end="\n")
gardientdescent_k_fold_cross_validation(X,y,10,weight_function_gradientdescent,0.000001)

(Mean Square Error for Test Set, Mean Square Error for Train Set):


(40.059119529357, 40.36672128230183)

**8.**

**Lasso Regression with Gradient Descent**

In [14]:
#Lasso Regression
def lasso_regression_fit(X, y, alpha=0.1, max_iter=1000, tol=1e-4, learning_rate=0.001):
    m, n = X.shape
    beta = np.zeros(n)
    
    for _ in range(max_iter):
        prev_beta = np.copy(beta)
        gradients = lasso_gradient(X, y, beta, alpha)
        beta -= learning_rate * gradients
        
        if np.linalg.norm(beta - prev_beta) < tol:
            break
            
    return beta

def lasso_gradient(X, y, beta, alpha):
    error = y - np.dot(X, beta)
    lasso_gradient = -(2 / y.size) * X.T.dot(error) + alpha * np.sign(beta)
    return lasso_gradient

def lasso_regression_predict(X, beta):
    return np.dot(X, beta)

def k_fold_cross_validation(X, y, k, alpha, learning_rate):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    mse_scores_for_test_set = []
    mse_scores_for_train_set = []
    
    for train_index, test_index in kf.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        beta = lasso_regression_fit(X_train, y_train, alpha=alpha, learning_rate=learning_rate)
        y_train_pred = lasso_regression_predict(X_train, beta)
        y_test_pred = lasso_regression_predict(X_test, beta)
        
        mse_train = np.mean((y_train - y_train_pred) ** 2)
        mse_test = np.mean((y_test - y_test_pred) ** 2)
        
        mse_scores_for_train_set.append(mse_train)
        mse_scores_for_test_set.append(mse_test)
    
    return np.mean(mse_scores_for_test_set), np.mean(mse_scores_for_train_set)

alpha = 0.1
learning_rate = 0.000001
k = 10

# Initialize and train Lasso regression model
mse_test, mse_train = k_fold_cross_validation(X_new, y, k, alpha, learning_rate)

print("Mean Square Error for Test Set:", mse_test)
print("Mean Square Error for Train Set:", mse_train)

Mean Square Error for Test Set: 77.98543188442451
Mean Square Error for Train Set: 76.99179204471955


**9.**

**Elastic Net Regression with gradient descent**

In [15]:
#Elastic Net Regression
def elastic_net_mse(y, y_predicted, weight, alpha1, alpha2):
    error = y - y_predicted
    
    l1_penalty = alpha1 * np.sum(np.abs(weight))
    
    l2_penalty = alpha2 * np.dot(weight, weight)
    
    return np.mean(error ** 2) + l1_penalty + l2_penalty

def elastic_net_gradient(X, y, y_pred, weight, alpha1, alpha2):
    error = y - y_pred
    
    l1_penalty_grad = alpha1 * np.sign(weight)
    
    l1_penalty_grad[0] = 0                                        # Exclude intercept term from L1 penalty gradient
    l2_penalty_grad = 2 * alpha2 * weight
    
    return -(2 / len(y)) * X.T.dot(error) + l1_penalty_grad + l2_penalty_grad

def elastic_net_gd(X, y, alpha1, alpha2, learning_rate, iterations=1000):
    weight = np.zeros(X.shape[1])
    
    for _ in range(iterations):
        y_pred = np.dot(X, weight)
        
        gradient = elastic_net_gradient(X, y, y_pred, weight, alpha1, alpha2)
        weight -= learning_rate * gradient
        
    return weight

def k_fold_cross_validation_elastic_net(X, y, alpha1, alpha2, learning_rate, k=10, iterations=1000):
    kf = KFold(n_splits=k, shuffle=True, random_state=42)
    
    mse_scores_test_set = []
    mse_scores_train_set = []
    
    for train, test in kf.split(X):
        
        X_train, X_test = X[train], X[test]
        y_train, y_test = y[train], y[test]
        
        weight = elastic_net_gd(X_train, y_train, alpha1, alpha2, learning_rate, iterations)
        
        y_train_pred = np.dot(X_train, weight)
        y_test_pred = np.dot(X_test, weight)
        
        mse_train = elastic_net_mse(y_train, y_train_pred, weight, alpha1, alpha2)
        mse_test = elastic_net_mse(y_test, y_test_pred, weight, alpha1, alpha2)
        
        mse_scores_train_set.append(mse_train)
        mse_scores_test_set.append(mse_test)
        
    return np.mean(mse_scores_test_set), np.mean(mse_scores_train_set)

alpha1 = 0.1
alpha2 = 0.2
learning_rate = 0.000001
k = 10
iterations = 1000

mse_test, mse_train = k_fold_cross_validation_elastic_net(X_new, y, alpha1, alpha2, learning_rate, k, iterations)
print("Mean Square Error for Test Set:", mse_test)
print("Mean Square Error for Train Set:", mse_train)

Mean Square Error for Test Set: 73.50806976253273
Mean Square Error for Train Set: 72.4778524881955


**10.**

*Given the option to predict future housing prices using one model, I would opt for polynomial transformation with ridge regression (referencing Question No. 6). This choice is based on its impressive performance, as evidenced by its lowest test mean squared error of 13.89. This suggests that the model exhibits low variance when applied to test data. Additionally, the average training mean squared error of 6.77 indicates that the model is not overly complex or prone to overfitting, meaning its bias is also low.

The current performance metrics are as follows:

Performance after polynomial transformation with the best Lambda (10.00):

-Average training MSE: 6.771
-Average test MSE: 13.894

These metrics were obtained using ridge regression (closed form) with the following parameters:

-Training dataset X, y and prediction dataset X(i)
-Regularization parameter (Lambda) set to 10
-Polynomial of degree 2
-Mean Squared Error (MSE) as the evaluation metric*
